**Install dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-yf0z7inc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-yf0z7inc
  Resolved https://github.com/huggingface/transformers.git to commit c8d98405a8f7b0e5d07391b671dcc61bb9d7bad5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install flash-attn --no-build-isolation


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.7 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.5-cp310-cp310-linux_x86_64.whl size=120352304 sha256=e70f2f0d4fae98c9ff9742404f3e067bbae56bd3212578ff81c30f73c5ae1a15
  Stored in directory: /root/.cache/pip/wheels/b2/67/52/8b6d5fcffdd9e1ec868f554cdef8f03eedb4bf4dcac852fca2
Successfully built flash-attn


In [ ]:
!pip install --upgrade optimum


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
pip show transformers

Name: transformers
Version: 4.39.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: optimum


In [ ]:
pip show optimum

Name: optimum
Version: 1.17.1
Summary: Optimum Library is an extension of the Hugging Face Transformers library, providing a framework to integrate third-party libraries from Hardware Partners and interface with their specific functionality.
Home-page: https://github.com/huggingface/optimum
Author: HuggingFace Inc. Special Ops Team
Author-email: hardware@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: coloredlogs, datasets, huggingface-hub, numpy, packaging, sympy, torch, transformers
Required-by: 


**Using the Whsiper large model**

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)

model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Defining the function for appending the textual data into the csv file**

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
def transcribe_and_append(audio_files, output_csv):
    if os.path.exists(output_csv) and os.stat(output_csv).st_size > 0:
        df = pd.read_csv(output_csv)
    else:
        df = pd.DataFrame(columns=["audio_path", "transcription"])

    # Iterating
    for audio_file in tqdm(audio_files, desc="Transcribing"):
        audio_path = os.path.join(audio_folder, audio_file)
        result = pipe(audio_path, generate_kwargs={"language": "english"})


        transcriptions = result["text"]


        transcription = " ".join(transcriptions)


        df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)


    df.to_csv(output_csv, index=False)
    print(f"Transcriptions saved to {output_csv}")

In [ ]:
audio_folder = "/content/drive/MyDrive/extracted_folder/segmented-sanket"
output_csv = "/content/drive/MyDrive/transcriptions33.csv"


In [ ]:

chunk_size = 100
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".mp3")]
for i in range(0, len(audio_files), chunk_size):
    chunk_files = audio_files[i:i+chunk_size]
    transcribe_and_append(chunk_files, output_csv)

Transcribing:   0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   1%|          | 1/100 [00:04<06:37,  4.01s/it]<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   2%|▏         | 2/100 [00:05<04:09,  2.55s/it]<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   3%|▎         |

Transcriptions saved to /content/drive/MyDrive/transcriptions33.csv


Transcribing:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   1%|          | 1/100 [00:06<10:39,  6.46s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"a

Transcriptions saved to /content/drive/MyDrive/transcriptions33.csv


Transcribing:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   1%|          | 1/100 [00:04<07:13,  4.38s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"a

...

Transcribing:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   1%|          | 1/100 [00:03<05:33,  3.37s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"a

Transcriptions saved to /content/drive/MyDrive/transcriptions33.csv


Transcribing:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   1%|          | 1/100 [00:04<07:18,  4.43s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"a

Transcriptions saved to /content/drive/MyDrive/transcriptions33.csv


Transcribing:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
<ipython-input-22-e525ba5783f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"audio_path": audio_path, "transcription": transcription}, ignore_index=True)
Transcribing:   1%|          | 1/100 [00:04<06:44,  4.09s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
<ipytho

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/transcriptions33.csv")

**Sorting all the audio files and the corresponding text in order for easy preprocessing**

In [ ]:
def get_sort_key(audio_path):
    parts = audio_path.split('/')
    file_name = parts[-1]
    parts = file_name.split('-')
    first_part = parts[0]
    second_part = parts[1].split('.')[0]
    return int(first_part), int(second_part)
df_sorted = df.iloc[df['audio_path'].apply(get_sort_key).argsort()]


In [ ]:
df_sorted.to_csv('metadata.csv')